In [1]:
import cv2
import numpy as np
import math
from collections import defaultdict


# reference : https://java2020.com/q/ktdcksju


def segment_by_angle_kmeans(lines, k=2, **kwargs):
    """Groups lines based on angle with k-means.

    Uses k-means on the coordinates of the angle on the unit circle 
    to segment `k` angles inside `lines`.
    """

    # Define criteria = (type, max_iter, epsilon)
    default_criteria_type = cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER
    criteria = kwargs.get('criteria', (default_criteria_type, 10, 1.0))
    flags = kwargs.get('flags', cv2.KMEANS_RANDOM_CENTERS)
    attempts = kwargs.get('attempts', 10)

    # returns angles in [0, pi] in radians
    angles = np.array([line[0][1] for line in lines])
    # multiply the angles by two and find coordinates of that angle
    pts = np.array([[np.cos(2*angle), np.sin(2*angle)]
                    for angle in angles], dtype=np.float32)

    # run kmeans on the coords
    labels, centers = cv2.kmeans(pts, k, None, criteria, attempts, flags)[1:]
    labels = labels.reshape(-1)  # transpose to row vec

    # segment lines based on their kmeans label
    segmented = defaultdict(list)
    for i, line in zip(range(len(lines)), lines):
        segmented[labels[i]].append(line)
    segmented = list(segmented.values())
    return segmented


def intersection(line1, line2):
    """Finds the intersection of two lines given in Hesse normal form.

    Returns closest integer pixel locations.
    See https://stackoverflow.com/a/383527/5087436
    """
    rho1, theta1 = line1[0]
    rho2, theta2 = line2[0]
    A = np.array([
        [np.cos(theta1), np.sin(theta1)],
        [np.cos(theta2), np.sin(theta2)]
    ])
    b = np.array([[rho1], [rho2]])
    x0, y0 = np.linalg.solve(A, b)
    x0, y0 = int(np.round(x0)), int(np.round(y0))
    return [[x0, y0]]


def segmented_intersections(lines):
    """Finds the intersections between groups of lines."""

    intersections = []
    for i, group in enumerate(lines[:-1]):
        for next_group in lines[i+1:]:
            for line1 in group:
                for line2 in next_group:
                    intersections.append(intersection(line1, line2)) 

    return intersections

def sort_vertex_clockwise(intersections):
    intersections_sorted = np.zeros([4, 2], dtype = np.float32)
    intersections.sort(key = lambda x : x[0][1]) # y값 기준 오름차순 정렬

    # 반으로 나눠서 크기 비교 두번씩
    if intersections[0][0][0] < intersections[1][0][0]:
        lu = intersections[0][0]
        ru = intersections[1][0]
    else:
        lu = intersections[1][0]
        ru = intersections[0][0]

    if intersections[2][0][0] < intersections[3][0][0]:
        rd = intersections[3][0]
        ld = intersections[2][0]
    else:
        rd = intersections[2][0]
        ld = intersections[3][0]


    intersections_sorted[0, :] = np.array(lu).astype(np.float32)
    intersections_sorted[1, :] = np.array(ru).astype(np.float32)
    intersections_sorted[2, :] = np.array(rd).astype(np.float32)
    intersections_sorted[3, :] = np.array(ld).astype(np.float32)
    
    return intersections_sorted


def perspective_trasform(src, w, h, intersections_sorted):
    # 좌상단부터 시계 방향으로 처리
    dst_pts = np.array([[0, 0],
                        [w - 1, 0],
                        [w - 1, h - 1],
                        [0, h - 1]]).astype(np.float32)
    pers_mat = cv2.getPerspectiveTransform(intersections_sorted, dst_pts) # M 구하기
    result = cv2.warpPerspective(src, pers_mat, (w, h)) # M 이용해 투시 변환
    
    return result


def main():
    
    # 1. BGR2Gray & Otsu
    src = cv2.imread('test_resize.png')
    cv2.imshow('src', src)
    src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    ret, otsu = cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 2. Morphology
    mask = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    erode = cv2.erode(otsu, mask, iterations = 1)
    dilate = cv2.dilate(erode, mask, iterations = 2) # remove noise (hole)

    # 3. edge, line detection
    edge = cv2.Canny(dilate, 50, 150)
    lines = cv2.HoughLines(edge, 1, math.pi / 180, 114) # threshold : 250 -> 114

    # 4. grouping by angle (vertical & horizontal)
    segmented = segment_by_angle_kmeans(lines)
    
    # 5. get intersections (4 vertex)
    intersections = segmented_intersections(segmented) # intersections[0][0] == [365, 252]
    
    # 6. get 4 vertexes' direction (clockwise)
    intersections_sorted = sort_vertex_clockwise(intersections)
    
    # 7. perspective transformation
    w = 400 
    h = 600
    result = perspective_trasform(src, w, h, intersections_sorted)

    cv2.imshow('result', result)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
    
    
main()

![](final_result.png)